# Extract EMS Data on EMSR586: Sydney Floods

This notebook shows how to acquire and parse flood metadata, and initial floodmaps, from the Copernicus Emergency Management Service web page. At the end of the notebook we will have selected the local government areas affected by flooding.

## Background

The Copernicus EMS Activation Mappings for individual events given by unique EMSR codes may be accessed [here](https://emergency.copernicus.eu/mapping/list-of-activations-rapid). Available resources include ZIP files contain multiple products (e.g., shapefiles) for an event.

**Update 2023: Recent website changes mean that the tables containing the EMSR information are displayed via an *iframe*.** 

The direct URL for these tables is [https://poc-d8.lolandese.site/search-activations1](https://poc-d8.lolandese.site/search-activations1)

The EMSR586 event covers flooding in NSW during early July 2022.
https://emergency.copernicus.eu/mapping/list-of-components/EMSR586

In [ ]:
# Necessary Python modules
import sys
import os
os.environ['USE_PYGEOS'] = '0'
import glob
from tqdm import tqdm
import numpy as np
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import timedelta
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping
from shapely.geometry import box
import ee
#import geemap.folium as geemap
import geemap.foliumap as geemap
import folium

import warnings
warnings.filterwarnings('ignore', '', RuntimeWarning)

from dotenv import load_dotenv

from ml4floods.data import ee_download, create_gt, utils
from ml4floods.data.copernicusEMS import activations

## Load environment and project details

As with the other notebooks, we load credentials and project details from a hidden ```.env``` file.

In [ ]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env"

# Uncomment for alternative version for Windows (r"" indicates raw string)
#env_file_path = r"C:/Users/User/floodmapper/.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] base path does not exist: \n{base_path} "
bucket_name = os.environ["BUCKET_URI"]
assert bucket_name is not None and bucket_name != "", f"Bucket name not defined {bucket_name}"
print("[INFO] Successfully loaded FloodMapper environment.")

**Set the details of the event and mapping session here.**

In [ ]:
# Official EMSR Code on Copernicus website
emsr_code = "EMSR586"

# Flooding date range (UTC)
# May need to start day or two earlier
flood_start_date = "2022-07-01"
flood_end_date = "2022-07-24"

## Download EMS Rapid Activation Data

First let's take a look at the recent Copernicus activations using the ML4Floods 'activations' module, which provides methods to query the EMS, download event data and parse information within.

**Note: ML4Floods code updated in late December 2023 with new EMSR scraping code.**

In [ ]:
## Fetch a table of activations events after a specified date
table_activations_ems = activations.table_floods_ems(event_start_date=flood_start_date)
table_activations_ems

We can see that information on EMSR587 is available, starting on 2022-07-03. Next we retrieve the URLs pointing to the activation products in ZIP files.

In [ ]:
# Retreive the URLs of available ZIP files for EMSR586
zip_files_activation_url_list = activations.fetch_zip_file_urls(emsr_code)
print("There are {:d} ZIP files available.".format(len(zip_files_activation_url_list)))
zip_files_activation_url_list.sort()
zip_files_activation_url_list

In [ ]:
# Specify the local folder to save into and create if necessary.
# By convention, we use the folder "<base_path>/flood-activations/<emsr_code>".
folder_out = os.path.join(base_path, "flood-activations", emsr_code, "ems_products").replace("\\", "/")
print("[INFO] Local output folder:\n", folder_out)
os.makedirs(folder_out, exist_ok=True)

Now download the files (if necessary) and unzip into folders.

In [ ]:
# Loop through the available files, download if necessary, and unzip
zip_files_activations = []
unzip_files_activations = []
print("[INFO] downloading {:d} files".format(len(zip_files_activation_url_list)))
for url_str in tqdm(zip_files_activation_url_list):
    zip_file_name = os.path.split(url_str)[-1]
    local_zip_file = os.path.join(folder_out, zip_file_name).replace("\\", "/")
    if not os.path.exists(local_zip_file):
        # Perform the download
        local_zip_file = activations.download_vector_cems(url_str, folder_out=folder_out)
    zip_files_activations.append(local_zip_file)
print("[INFO] unzipping {:d} files".format(len(zip_files_activation_url_list)))
for local_zip_file in tqdm(zip_files_activations):
    unzipped_file = activations.unzip_copernicus_ems(local_zip_file, folder_out=folder_out)
    unzip_files_activations.append(unzipped_file)
    

In [ ]:
# Check that the files have been downloaded and unzipped
cmd = f"tree -L 1 {folder_out}"
!{cmd}

Each folder contains shapefiles with useful information on:

* facilitiesL = linear features (power lines)
* facilitiesA = facilities (parks, golf courses, plants)
* builtUpA = build up areas (urban areas)
* transportationL = transportations lines (e.g., roads and tracks)
* transportationA = transportation areas (e.g., airports)
* physiographyL = physiography lines (e.g., height contours)
* naturalLandUseA = land use areas
* imageFootprintA = source image footprint

However, the information we care about is in these files:

* observedEventA = observed event areas (polygons)
* hydrographyL = linear water bodies (permanent streams)
* hydrographyA = wide water bodies (permanent lakes, rivers)
* areaOfInterest = polygon around AoI

## Building EMSR Flood Metadata and Floodmaps


The function `filter_register_copernicusems` firsts checks that all the shapefiles follow the expected conventions with respect to timestamp and data availability. It then extracts 1) the AoI polygons, 2) hydrography information, 3) event information and 4) metadata into a separate dictionary for each ZIP file. 

The function `generate_floodmap` creates a geoDataFrame for each ZIP file containing the EMS-derived hydrography and flooding masks.

All of this information is appended to an in-memory list variable called `registers`.

In [ ]:
# Extract the event start date from the table
code_date = table_activations_ems.loc[emsr_code]["CodeDate"]

# Loop through the unzipped folders and extract the metadata and water masks into memory
registers = []
for unzip_folder in unzip_files_activations:
    folder = os.path.split(unzip_folder)[-1]
    
    # Extract 1) AOI, 2) hydrography, 3) observed event into a dict
    metadata_floodmap = activations.filter_register_copernicusems(unzip_folder, code_date)
    
    # Create a 'floodmap' hydrography dataframe and save to list of dictionaries called 'registers'
    if metadata_floodmap is not None:    
        floodmap = activations.generate_floodmap(metadata_floodmap, folder_files=unzip_folder)
        registers.append({"metadata_floodmap": metadata_floodmap, "floodmap": floodmap})
        print(f"Folder {folder} processed correctly.")
    else:
        print(f"Folder {folder} does not follow the expected format. It won't be processed")

In [ ]:
# Sanity check
print("[INFO] There are {:d} registers available.".format(len(registers)))

## Visualise the hydrology information from the first ZIP file

In [ ]:
# Examine the metadata associated with the first ZIP file
idx = 0
metadata = registers[idx]['metadata_floodmap']
metadata

In [ ]:
# Examine the hydrology information associated with the first file
floodmap = registers[idx]['floodmap']
floodmap

In [ ]:
# Check the unique classes of data in the floodmap table
np.unique(floodmap.source)

These classes are:
 * area_of_interest = bounds of the area
 * flood = bounds of the flooded areas
 * hydro = lakes and rivers
 * hydro_l = linear water courses (negligible width)

In [ ]:
# Plot the flood
fig, ax = plt.subplots(figsize=(16,16))
floodmap[floodmap["source"] == "area_of_interest"].plot(ax=ax, facecolor="None", edgecolor="yellow",
                                               label="AOI", linewidth=0.8)
floodmap[floodmap["source"] == "flood"].plot(ax=ax, facecolor="blue", edgecolor="None", 
                                             label="Flood Maps", linewidth=0.8)
floodmap[floodmap["source"] == "hydro_l"].plot(ax=ax, facecolor="None", edgecolor="red", 
                                               label="Hydrography Line", linewidth=0.8)
floodmap[floodmap["source"] == "hydro"].plot(ax=ax, facecolor="magenta", edgecolor="None",
                                               label="Hydrography Area", linewidth=0.8)
ax.set(title="Hydrography Map")
plt.show()

## Merge the hydrology from all ZIP files and visualise

First merge the AoI and hydrology information into single dataframes.

In [ ]:
aoi_list = []
floodmap_list = []
for register in registers:
    # Build a list of AoI polygons
    aoi_list.append(register["metadata_floodmap"]["area_of_interest_polygon"])
    # Build a list of water polygons
    floodmap_list.append(register["floodmap"])

# Create a geodataframe of the AoIs and the external boundary
crs_str = register["metadata_floodmap"]["reference system"]
aoi_df = gpd.GeoDataFrame(geometry=aoi_list, crs=crs_str)
aoi_df["AOI_Index"] = aoi_df.index
aoi_ext_poly = box(*aoi_df.total_bounds)
aoi_ext_df = gpd.GeoDataFrame(geometry=[aoi_ext_poly], crs=crs_str)

# Create a dataframe of the water components
floodmap_all = pd.concat(floodmap_list)

Then plot all information on a common map (may take 30 - 60 seconds).

You probably don't want to do this for larger areas than shown here due to processing limits.

In [ ]:
# Plot the information in the EMSR ZIP files
fig, ax = plt.subplots(figsize=(16,16))
floodmap_all[floodmap_all["source"] == "flood"].plot(ax=ax, facecolor="blue", edgecolor="None", 
                                             label="Flood Maps", linewidth=0.1)
floodmap_all[floodmap_all["source"] == "hydro_l"].plot(ax=ax, facecolor="None", edgecolor="red", 
                                               label="Hydrography Line", linewidth=0.1)
floodmap_all[floodmap_all["source"] == "hydro"].plot(ax=ax, facecolor="magenta", edgecolor="None",
                                               label="Hydrography Area", linewidth=0.1)
aoi_df.plot(ax=ax, facecolor="None", edgecolor="black", 
                                               label="AoI", linewidth=0.1)
aoi_ext_df.plot(ax=ax, facecolor="None", edgecolor="black", 
                                               label="AoI External", linewidth=0.1)
ax.set(title="Water Maps")
plt.show()

## Determine which LGAs are affected by flooding

In [ ]:
# Load the LGAs from the local shapefile
# We could also perform a query on the database
lga_file_path = os.path.join(base_path, "resources/LGAs/LGA_2022_AUST_GDA2020.shp")
lga_gdf = gpd.read_file(lga_file_path).to_crs(crs_str)
lga_gdf.dropna(inplace=True)
lga_gdf.plot()

In [ ]:
# Get the unique names of the intersecting LGAs
lga_list = lga_gdf.overlay(aoi_df, how = 'intersection')["LGA_NAME22"].to_list()
lga_list = list(set(lga_list))

# Filter the LGA dataframe for the affected LGA names
lga_affected_df = lga_gdf[lga_gdf["LGA_NAME22"].isin(lga_list)]

print(f"The {len(lga_list)} affected LGAs are:\n", lga_list)

In [ ]:
# And visualise the affected LGAs
m = lga_affected_df.explore(style_kwds={"fillOpacity": 0.0}, name="Affected LGAs")
aoi_df.explore(m=m, style_kwds={"fillOpacity": 0.1}, color="red", name="AoIs")
folium.LayerControl(collapsed=False).add_to(m)
m

Note that the list of LGA names can be supplied as an argument to the Floodmapper data download script ```01_download_images.py```. We can format that list using a simple Python statement.

In [ ]:
# Print a comma-separated list of LGA names
",".join(lga_list)

## Choose the area to map

From a visual inspection of the map, we choose to map the LGAs intersected by AoIs 30 and 35, covering flooding on the Hunter river. These are: 

```
['Newcastle', 'Port Stephens', 'Maitland', 'Singleton', 'Cessnock']
```

We will select these geometries and save to the local disk as a GeoJSON file for use with NEMA Floodmapper. Of course, we could also simply use all of the LGAs in ```lga_afffected_df```.

In [ ]:
# Select the subset of LGAs to map
lga_list = ['Newcastle', 'Port Stephens', 'Maitland', 'Singleton', 'Cessnock']
lga_selected_df = lga_gdf[lga_gdf["LGA_NAME22"].isin(lga_list)]
lga_selected_df.plot()

In [ ]:
# Write the selected LGA shapes to local disk
folder_out = os.path.join(base_path, "flood-activations", emsr_code).replace("\\", "/")
path_out = os.path.join(folder_out, "LGAs_To_Map.geojson").replace("\\", "/")
os.makedirs(folder_out, exist_ok=True)
lga_selected_df.to_file(path_out, driver="GeoJSON", crs="EPSG:4326")
print("[INFO] Wrote file to:\n", path_out)

Alternatively, we could choose some of the EMSR AoIs to map instead:

In [ ]:
# Write selected EMSR AoI shapes to disk
path_out = os.path.join(folder_out, "EMSR_AoIs_To_Map.geojson").replace("\\", "/")
aoi_selected = aoi_df[(aoi_df.index == 30) | (aoi_df.index == 35)]
aoi_selected.to_file(path_out, driver="GeoJSON")
print("[INFO] Wrote file to:\n", path_out)

In [ ]:
# Check if the files appear on disk
!ls {folder_out}

In the next notebook, we will split these shapes into square processing patches for use with the FloodMapper system.